# RTML Lab 11: GNNs

Today we explore GNNs, in particular [Graph Convolutional Networks by Kipf and Welling (2017)](https://arxiv.org/abs/1609.02907).

In this lab, we will develop Graph Neural Networks (GNN).

Some references:
- [PyTorch Geometric tutorial for GCN](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html)
- [Graph Neural Networks: A review of methods and applications](arxiv.org/ftp/arxiv/papers/1812/1812.08434.pdf)
- [A Gentle Introduction to Graph Neural Networks (Basics, DeepWalk, and GraphSage)](https://towardsdatascience.com/a-gentle-introduction-to-graph-neural-network-basics-deepwalk-and-graphsage-db5d540d50b3)
- [Hands-on Graph Neural Networks with PyTorch & PyTorch Geometric](towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8)
- [Graph Neural Network (GNN): What It Is and How to Use It](https://builtin.com/data-science/gnn)
- [An Introduction to Graph Neural Network(GNN) For Analysing Structured Data](https://towardsdatascience.com/an-introduction-to-graph-neural-network-gnn-for-analysing-structured-data-afce79f4cfdc)

## What are GNNs?

Graph neural networks (GNNs) were first introduced in 2009,
[cite](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.1015.7227&rep=rep1&type=pdf).
A GNN is a neural model that can be applied directly to graphs without prior knowledge of each component.
GNN provides a convenient method for performing node-, edge-, and graph-level prediction tasks.

GNNs have recently received a lot of attention because of their
ability to analyze graph structured data such as a set of objects and their relationships.

In order to incorporate graph-structured information, in a data processing step,
the underlying graph structure is encoded using the topological relationships between the nodes of the graph.
Graph models include recursive neural networks and Markov chains, which are commonly used to solve graph and node-focused problems.

## Graphs

A graph $G=(V,E)$ is a data structure made up of two parts: a set of *vertices* ($V$) and a set of *edges* ($E$).
A graph is a mathematical structure that is used to examine the pair-wise relationship between objects and entities.
Edges can be either *directed* or *undirected*, depending on whether there exist directional dependencies between the vertices.
Here's an example graph:

<img src="img/GNNSimpleGraph.webp" title="A simple Graph" style="width: 150px;" />

Besides the set notation mentioned above,
a graph is often represented by an adjacency matrix $A$.
If a graph has $N$ nodes, then $A \in \mathbb{R}^{N\times N}$.
Mathematically, the graph's adjacency matrix has a value of 1 at element $(i,j)$ only when there is an edge
from node $i$ to node $j$;
otherwise the value should be zero. Here's an example for the a directed graph in which the edges have directionality:

<img src="img/AdjacencyMatrix.PNG" title="Adjacency matrix" style="width: 600px;" />

Another matrix, the *feature matrix* $X$ is sometimes provided to describe the nodes in the graph.
If each node has $C$ features, then $X \in \mathbff{R}^{N \times C}$.

<img src="img/FeatureMatrix.PNG" title="Feature matrix" style="width: 600px;" />

Interestingly, we can perform the multiplication $H = AX$. Each row $i$ of the result matrix $H$
is the summation of the features of all nodes $j$ connected to node $i$.

<img src="img/multiMatrix.PNG" title="Summation matrix" style="width: 600px;" />

## Graphs represent non-Euclidean data

A graph cannot be represented in a Euclidean coordinate system.
This makes graph data more difficult to interpret than other types of data such as waves, images, and time-series signals.
These types of data can all be represented in a 2D or 3D space.
Another issue is that different looking graphs can have the same
intrinsic structure. For example the following two graphs have the same adjacency matrix:

<table><tr>
<td> <img src="img/GraphA.webp" title="A" style="width: 150px;" /> </td>
<td> <img src="img/GraphB.webp" title="B" style="width: 150px;" /> </td>
</tr></table>

We could also renumber the nodes. That would change the adjacency matrix but the graph would actually intrinsically be the same.
Finally, large graphs are not easy to visualize. How about this one?

<img src="img/circuitnetlist.webp" title="B" style="width: 300px;" />

## Graphs are natural representations of some kinds of data

Despire these difficulties, graphs are ideal natural representations of relationships and interactions between entities, such
as relationships between people in a social context (social network analysis or SNA).

## Traditional graph analysis algorithms

Computer scientists have studied graphs for a long time. There are existing graph algorithms for
1. Search (depth first search and breadth first search)
2. Shortest path finding (Dijkstra’s algorithm)
3. Spanning-tree finding (Prim’s algorithm)
4. Clustering (k-Means, etc.)

However, here, we are interested in performing machine learning over graph structures. These are
relatively new algorithms.

## Basic graph neural networks

A GNN is a type of neural network that operates directly on
graph structures. Node classification is one common application of GNNs.
In a node classification problem, each node has a label, and we want to predict
the label of each node without using the ground truth.

RNNs and CNNs can be thought of as GNNs.
There are several other GNNs attracing attention today:
1. Graph convolutional networks (GCNs)
2. Graph attention networks (GATs)

Here we'll give the idea of the simplest form of GNN using message passing.

### GNN message passing

Each node $i$ in a node classification problem is
identified by feature vector $x_i$ and associated with a ground truth label
$y_i$. The goal is to use the labeled nodes in a partially labeled graph $G$ to predict the labels of the unlabeled nodes.
We may have multiple layers of processing in which we transform
$x_i$ to a hidden representation $h_i$ containing
information about node $i$'s neighbors. That is,

$$h_i=f(x_i,x_{co[i]},h_{ne[i]},x_{ne[i]}),$$

where $x_{co[i]}$ refers to the features of the edges connecting with node $i$, $h_{ne[i]}$ refers to the hidden representations
of node $i$'s neighboring nodes, and $x_{ne[i]}$ refers to the features of node
$i$'s neighboring nodes.

It turns out that if $f()$ is simple enough,
we can use the Banach fixed point theorem to rewrite the above equation as an iteratively updated process that
gives a unique solution for the $h_i$. This iterative operation is also known as *message passing* or *neighborhood aggregation*.
Concatenating the $x_i$ and $h_i$ we can write $f$ as
an operation $F$ over matrices:
$$H^{t+1}=F(H^t,X).$$

### Output

The output of the GNN depends on the hidden representations $h_i$ as well
as (considering a "skip" connection) the feature vectors $x_i$.
This can be implemented by a function $g$:

$$o_i=g(h_i,x_i)$$

Both $f$ and $g$ can be implemented by feed-forward fully-connected neural networks. 

### Loss function

Depending on the type of inference being performed, we may use L1, L2, or cross entropy loss at the output.
Any of these loss functions can be optimized using gradient descent.

### Improvements to the basic GNN

There are many variations on the basic GNN idea. Among the most successful approaches, Kipf and Welling's (2017) GCN
uses a first-order approximation to message passing using a simplified form of graph spectral convolutions that are
stacked into multiple layers in order to accomplish the transformation of
the input features to obtain hidden layer representations that incororate information from neighboring nodes.


## DeepWalk

Reference: http://www.perozzi.net/publications/14_kdd_deepwalk.pdf

DeepWalk was
the first algorithm to perform unsupervised node embedding learning.
It is trained similarly to a word embedding model.
The motivation is that the distribution of nodes in a graph under a random walk of the nodes and
words in a corpus both follow a power law, as illustrated by the figure in the paper:

<img src="img/deepwalk.webp" title="http://www.perozzi.net/publications/14_kdd_deepwalk.pdf" style="width: 800px;" />

The algorithm involves two steps:

1. Perform random walks on nodes in the graph to generate node sequences
2. Run a skip-gram algorithm to learn an embedding of each node based on the node sequences generated in step 1

At each time step of the random walk, the next node is sampled uniformly from the neighbors of the previous node. Each sequence is then truncated into
sub-sequences of length $2|w| + 1$, where $w$ denotes the window size in the skip-gram method.

The skip-gram method is explained well in [this article from Towards Data Science](https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c).

A hierarchical softmax is performed to reduce the cost of the softmax over the large number of nodes.
In an ordinary softmax, we must compute $e^{x_k}$ for every index $k$:

$$softmax(x)_i = \frac{e^{x_i}}{\sum_{k=1}^K e^{x_k}}$$

The computation time is $O(|V|)$. Hierarchical softmax imposes a binary tree over the vertices.
Each inner node contains a binary classifier that determines which path to take. To compute the probability of a given vertex $v_k$, we simply add
the probabilities of each sub-path from the root node to the leaf $v_k$. Because the probability of each node's children sums to one,
the property that the sum of the probability of all vertices equals one remains valid in the hierarchical softmax.
As the longest path for an element, the computation time is now reduced to $O(\log|V|)$.

<img src="img/HierarchicalSoftmax.webp" title="http://www.perozzi.net/publications/14_kdd_deepwalk.pdf" style="width: 500px;" />

After training a DeepWalk GNN, the model has learned a good representation of each node, as shown in the figure below. In the input graph, different colors represent different labels. We can see in the output graph (embedding with two dimensions) that nodes with the same labels are clustered together, whereas most nodes with different labels are properly separated.

<img src="img/Deepwalkresult.webp" title="http://www.perozzi.net/publications/14_kdd_deepwalk.pdf" style="width: 500px;" />

The main problem with DeepWalk is that it lacks generalization ability. When a new node is added, the model must be re-trained to represent this node (transductive). As a result, such GNN is unsuitable for dynamic graphs with constantly changing nodes.

## GraphSage

Reference: https://www-cs-faculty.stanford.edu/people/jure/pubs/graphsage-nips17.pdf

GraphSage allows induction by inductively learning an embedding for each node.
In particular, each node is represented by the sum of its neighbors' features.
As a result, even if a previously unseen node appears in the graph during training,
it can still be properly represented by its neighbors.

<img src="img/GraphSageAlgo.webp" title="http://www.perozzi.net/publications/14_kdd_deepwalk.pdf" style="width: 500px;" />

The number of update iterations is indicated by the outer loop, and $h^k_v$ is the latent vector of node $v$ at update iteration $k$. $h^k_v$ is updated at each update iteration using an aggregation function, the latent vectors of $v$ and $v$'s neighborhood from the previous iteration, and a weight matrix $W^k$. There are three aggregation functions as:

1. **Mean aggregator**: takes the average of the latent vectors of a node and all its neighborhood.

$$h_v^k\leftarrow \sigma(W\cdot \text{MEAN}(\{h_v^{k-1}\}\cup \{h_u^{k-1},\forall u \in \mathcal{N}(v)\}))$$

Compared with the original equation, it removes the concatenation operation at line 5 in the above pseudo code. This operation can be viewed as a “skip-connection”, which later in the paper proved to largely improve the performance of the model.

2. **LSTM aggregator**: Since the nodes in the graph don’t have any order, they assign the order randomly by permuting these nodes.

3. **Pooling aggregator**: This operator performs an element-wise pooling function on the neighboring set. Below shows an example of max-pooling:

$$\text{AGGREGATE}_k^{pool}= \max(\{\sigma(W_{pool}h_{u_i}^k+b),\forall u_i \in \mathcal{N}(v)\})$$

which can be substituted for mean-pooling or any other symmetric pooling function. It is stated that the pooling aggregator performs the best, while the mean-pooling and max-pooling aggregators perform similarly. The paper's default aggregation function is max-pooling.

### Loss function

The loss function is defined as the following:

$$J_{\mathcal{G}}(z_u)=-\log(\sigma(z_u^\top))-Q\cdot \mathbb{E}_{v_n\sim P_n(v)}\log(\sigma(-z_u^\top z_{v_n}))$$

where $u$ and $v$ co-occur in a fixed-length random walk, while $v_n$ are the negative samples that don’t co-occur with $u$. Such loss function encourages nodes closer to have similar embedding, while those far apart to be separated in the projected space. Via this approach, the nodes will gain more and more information about their neighborhoods.

By aggregating its nearby nodes, GraphSage generates representable embedding for unseen nodes. It enables the application of node embedding to domains involving dynamic graphs, where the structure of the graph is constantly changing.

## Spectral graph convolution and GCNs

See the lecture notes from class.

## GNN problems

GNNs are generally used for three tasks:

- Node classification
- Link prediction
- Graph classification

### Node classification

The goal of node classification is to predict the node embedding for each node in a graph. This type of problem is typically trained semi-supervised, with only a portion of the graph labeled. Citation networks, Reddit posts, YouTube videos, and Facebook friendships are examples of common node classification applications.

### Link prediction

The task of link prediction is to understand the relationship between entities in graphs and predict whether two entities are connected. A recommender system, for example, can be modeled as a link prediction problem. When we feed the model a collection of user reviews of various products, the task is to predict the users' preferences and tune the recommender system to push more relevant products based on the users' interests.

### Graph classification

The goal of graph classification is to divide the entire graph into different categories. It's similar to image classification, but the target is a graph. There are numerous industrial problems where graph classification can be used; for example, in chemistry, biomedicine, or physics, we can provide the model with a molecular structure and ask it to classify the target into meaningful categories. The model then speeds up the analysis of atoms, molecules, and other structured data.

## GNN Applications

### Natural Language Processing (NLP)

GNN is frequently used in natural language processing (NLP), which is where GNN got its start. If you've worked with natural language processing (NLP), you're probably thinking that text is a type of sequential or temporal data that we can describe with a recurrent neural network (RNN) or a long short-term memory (LTSM). GNN, on the other hand, approaches the problem from an entirely different perspective. GNN predicts categories by utilizing the inner relationships of words or documents. A citation network, for example, attempts to predict each paper's label in a network based on the paper citation relationship and the words cited in other papers. GNN can also construct a syntactic model by examining different parts of sentences rather than only working sequentially as RNN or LTSM do.

### Computer Vision

Many GNN-based methods have achieved cutting-edge performance in image object detection, but we still don't know the relationships between the objects. Using graphs to model relationships between objects detected by a CNN-based detector is one successful application of GNN in computer vision (CV). After detecting objects in images, they are fed into a GNN inference for relationship prediction. The GNN inference produces a generated graph that models the relationships between various objects.

<img src="img/16_gnn.png" title="" style="width: 500px;" />

Image generation from graph descriptions is another intriguing CV application. This is almost the inverse of the preceding application. Text-to-image generation using generative adversarial network (GAN) or autoencoder is the traditional method of image generation. Rather than using text to describe images, graph-to-image generation provides more information about the semantic structures of the images.

<img src="img/17_gnn.png" title="" style="width: 500px;" />

The most intriguing application is zero-shot learning (ZSL), which learns to classify an object with no target class training samples. If no training samples are provided, the model must think in order to recognize a target. Assume we're given three images and told to find an okapi among them. We've never seen an okapi before, but if we're also told that an okapi is a deer-faced animal with four legs and zebra-striped skin, it's not difficult to figure out which one is an okapi. The detected features are typically converted into text to simulate this thought process. Text encodings, on the other hand, are independent of one another. The relationships between the text descriptions are difficult to model. Graph representations, on the other hand, accurately model these relationships and assist the machine in thinking more like a human.

<img src="img/18_gnn.png" title="" style="width: 500px;" />

### Other applications

Human behavior detection, traffic control, molecular structure study, recommender systems, program verification, logical reasoning, social influence prediction, and adversarial attack prevention are some of the more practical applications of GNN. Through social network analysis, for example, GNN can be used to cluster people into different community groups.

## Setup for PyTorch Geometric (PyG)

PyG only seems to work with recent versions of PyTorch, so you may need to upgrade:

In [ ]:
import os

# Set proxy for AIT

os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

# Install newer torch version and PyG

!pip install --upgrade torch
!pip install torch_geometric


Be sure to restart your kernel if you're doing this in Jupyter.

If you want to use some of the datasets that involve large sparse matrices, you may need
a proper installation of the `pytorch-sparse` package. As of the time of writing, we found
the following to work:

    pip3 install torch==1.13.1+cu117 torchvision==0.14.1+cu117 \
                 torchaudio==0.13.1 torch_geometric --extra-index-url https://download.pytorch.org/whl/cu117
    pip3 install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu117.html

The main thing seems to be that the versions of `torch-scatter` and `torch-sparse` need to
match your PyTorch and CUDA versions.

## Our first graph

OK, with that set up, let's use PyG to build a graph:

In [1]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)


In [3]:
data

Data(x=[3, 1], edge_index=[2, 4])

So we see that this defines a graph with three nodes and two edges. The edge index
is in so-called [COO format](https://pytorch.org/docs/stable/sparse.html#sparse-coo-docs) (a format for representing sparse matrices)
and in this case is a $2\times 2|E|$ tensor.

The indices are indexing the set of nodes, in the range 0 to $N-1$.

The following is equivalent. Note that each undirected edge is denoted by two pairs of indices:

In [ ]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)

The tensor `x` is providing the *features* of the nodes in the graph, in this case a scalar for each node. As we know,
node features can be vectors as well, but all nodes must have the same feature dimensionality.

We can check the validity of our graph with the `validate()` method:

In [4]:
data.validate(raise_on_error=True)

True

Try some other attributes and:
- `x`
- `edge_index`
- `edge_attr`
- `num_nodes`
- `num_edges`
- `num_node_features`
- `has_isolated_nodes()`
- `has_self_loops`
- `is_directed`

And you can move a graph to a GPU just like a tensor:

In [5]:
device = torch.device('cuda:0')
data = data.to(device)

## PyG datasets (Cora)

Let's load the Cora dataset:

In [6]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./data/Cora', name='Cora')

Processing...
Done!


Try a few of the dataset's methods:
- `len(dataset)`
- `dataset.num_classes`
- `dataset.num_node_features`
- `dataset[0]`

From this we see that Cora is a single graph.
We will only be doing transduction with this dataset.
After assigning the single graph to a variable such as `data`,
explore its characteristics:
- `data.is_undirected()`
- `data.train_mask.sum().item()`
- `data.val_mask.sum().item()`
- `data.test_mask.sum().item()`

As this graph is mainly used to test semi-supervised learning algorithms, we see that the validation and test data are more numerous than the training data.


## Mini-batches

How can we train on multiple examples in parallel? PyG puts multiple graphs together with block-diagonal adjacency matrices and concatenated features.
See the [tutorial section on mini-batches](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html#mini-batches) for more
information.

For a GCN on Cora, however, we don't need fancy data loaders or parallel mini-batches, as we're dealing with just one graph.

## Define a GCN

So let's define a GCN for Cora, then:

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

Moving the model to a GPU and training is the same as for any other PyTorch model. The "data" is treated as a single example:

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

After training, we an evaluate on the test dataset:

In [10]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7950


## Exercises

1. Implement early stopping using the validation set for the Cora GCN.
2. Reproduce the rest of the results from Kipf and Welling with your GCN.
3. Implement the [Graph Attention Network](https://arxiv.org/abs/1710.10903) and reproduce their results on the datasets available via PyG.


In [ ]:
from torch_geometric.datasets import nell

dataset = nell.NELL(root='./data/NELL')
